In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Reshape, Flatten, BatchNormalization, MaxPooling2D
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical

In [3]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [4]:
def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [5]:
def prepare_data(train_data, val_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    val_labels = encoder.transform(val_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [7]:
# import train data
adl_1_1 = pd.read_csv("./reduced_dataset/ACC/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("./reduced_dataset/ACC/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("./reduced_dataset/ACC/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("./reduced_dataset/ACC/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("./reduced_dataset/ACC/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("./reduced_dataset/ACC/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("./reduced_dataset/ACC/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("./reduced_dataset/ACC/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("./reduced_dataset/ACC/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("./reduced_dataset/ACC/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("./reduced_dataset/ACC/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("./reduced_dataset/ACC/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("./reduced_dataset/ACC/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("./reduced_dataset/ACC/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("./reduced_dataset/ACC/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("./reduced_dataset/ACC/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("./reduced_dataset/ACC/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("./reduced_dataset/ACC/ADL5Opportunity_taskB2_S3.csv",header=None)

In [11]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={9: 'labels'}, inplace =True)
val_data.rename(columns ={9: 'labels'}, inplace =True)
test_data.rename(columns ={9: 'labels'}, inplace =True)

In [12]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 10), val (60949, 10), test (118750, 10)


In [13]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [14]:
print(scaled_train.shape)
print(train_labels.shape)

(497014, 9)
(497014, 18)


In [16]:
num_sensors = 9
window_size = 24
step_size = 12
classes = 18

In [17]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [18]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [108]:
model = Sequential()
inputshape = (window_size, num_sensors, 1)

model.add(Flatten(input_shape=inputshape))

model.add(Dense(128,kernel_initializer='glorot_normal', activation='tanh',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer_1'))

model.add(Dropout(0.5, name='1_dropout_layer'))

model.add(Dense(32,kernel_initializer='glorot_normal', activation='tanh',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer_2'))

model.add(Dropout(0.5, name='2_dropout_layer'))

model.add(Dense(128,kernel_initializer='glorot_normal', activation='tanh',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer_3'))

model.add(Dropout(0.5, name='3_dropout_layer'))

model.add(Dense(216,kernel_initializer='glorot_normal', activation='sigmoid',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer_4'))
model.add(Reshape((window_size, num_sensors,1)))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=opt)

In [92]:
model = Sequential()

dropout_prob = 0.8
inputshape = (window_size, num_sensors, 1)

In [94]:
model.add(Conv2D(10, kernel_size=(5,1), strides=1,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))

In [95]:
model.add(Conv2D(10, kernel_size=(5,1), strides=1,
                 kernel_initializer='glorot_normal', name='2_conv_layer'))

In [96]:
model.add(Conv2D(20, kernel_size=(5,3), strides=(1,3),
                 activation='tanh',
                 kernel_initializer='glorot_normal', name='3_conv_layer'))

In [97]:
model.add(MaxPooling2D(pool_size=(2,1)))

In [98]:
model.add(Flatten())

In [99]:
model.add(Dense(128,kernel_initializer='glorot_normal', activation='tanh',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer_1'))

model.add(Dropout(dropout_prob, name='1_dropout_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal', activation='tanh',
                bias_initializer=initializers.Constant(value=0.1), name='red_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(Dense(128,kernel_initializer='glorot_normal', activation='tanh',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer_2'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=opt)

In [106]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

flatten_14: input shape: (None, 24, 9, 1) output shape: (None, 216)
dense_layer_1: input shape: (None, 216) output shape: (None, 128)
1_dropout_layer: input shape: (None, 128) output shape: (None, 128)
dense_layer_2: input shape: (None, 128) output shape: (None, 128)
2_dropout_layer: input shape: (None, 128) output shape: (None, 128)
dense_layer_3: input shape: (None, 128) output shape: (None, 128)
3_dropout_layer: input shape: (None, 128) output shape: (None, 128)
dense_layer_4: input shape: (None, 128) output shape: (None, 216)
reshape_3: input shape: (None, 216) output shape: (None, 24, 9, 1)


In [109]:
batchSize = 200
train_epoches = 50

model.fit(reshaped_train,reshaped_train,epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_reduced.h5')

Epoch 1/50
41416/41416 [==============================] - 3s 72us/step - loss: 0.0087
Epoch 2/50
41416/41416 [==============================] - 2s 45us/step - loss: 0.0020
Epoch 3/50
41416/41416 [==============================] - 2s 47us/step - loss: 0.0015
Epoch 4/50
41416/41416 [==============================] - 2s 45us/step - loss: 0.0013
Epoch 5/50
41416/41416 [==============================] - 2s 45us/step - loss: 0.0012
Epoch 6/50
41416/41416 [==============================] - 2s 45us/step - loss: 0.0011
Epoch 7/50
41416/41416 [==============================] - 2s 45us/step - loss: 0.0011
Epoch 8/50
41416/41416 [==============================] - 2s 45us/step - loss: 0.0010
Epoch 9/50
41416/41416 [==============================] - 2s 45us/step - loss: 9.8557e-04
Epoch 10/50
41416/41416 [==============================] - 2s 44us/step - loss: 9.6347e-04
Epoch 11/50
41416/41416 [==============================] - 2s 45us/step - loss: 9.4376e-04
Epoch 12/50
41416/41416 [===============

ValueError: Error when checking target: expected reshape_4 to have 4 dimensions, but got array with shape (9894, 18)

In [116]:
from keras.models import Model
from sklearn.decomposition import PCA

layer_name = 'dense_layer_2'
inter_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

network_output = inter_layer_model.predict(reshaped_train)

# PCA with certain number of principal components
pca = PCA(n_components=10)
pca.fit(network_output)
new_output = pca.transform(network_output)

In [117]:
print(network_output)

[[ 0.21964681 -0.34140134 -0.36596084 ..., -0.21540095 -0.01774176
  -0.03950792]
 [ 0.21112008 -0.38408151 -0.40389311 ..., -0.20702547 -0.01994131
  -0.02055739]
 [ 0.1735802  -0.40010458 -0.4214659  ..., -0.16527715 -0.00306433
  -0.0029956 ]
 ..., 
 [ 0.12283116  0.14538564  0.0953061  ..., -0.14268973 -0.05730413
  -0.30291271]
 [ 0.13366476  0.13920215  0.08839852 ..., -0.15403502 -0.05867961
  -0.30370635]
 [ 0.14445229  0.1330096   0.08148516 ..., -0.1653261  -0.06004905
  -0.30449286]]


In [118]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

svm_labels = np.argmax(train_labels, axis=1)

# train a one-vs-one multi-class support vector machine
clf = svm.SVC(decision_function_shape='ovr')
clf.fit(new_output, svm_labels)

# predict test data
svm_pred = clf.predict(pca.transform(inter_layer_model.predict(reshaped_val)))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(val_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

scores = cross_val_score(clf, pca.transform(inter_layer_model.predict(reshaped_val)),new_test_labels, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

[ 0.81231672  0.81630648  0.81710914  0.82195846  0.81962339]
Accuracy: 0.82 (+/- 0.01)
Test accuracy is: 0.8156754627806223
The f1-score with sklearn function is 0.7349925970478752
Average f1-score is 0.049915642323451435


C:\Users\Tommy Azzino\Anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Tommy Azzino\Anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))

Standard accuracy is 0.8603193854861533
0 0.832524762482 0.931251512951
1 0.0058621386699 0.608695652174
2 0.00960177885587 0.883720930233
3 0.00606428138266 0.59872611465
4 0.00838892257934 0.847457627119
5 0.0230442692541 0.547738693467
6 0.0161714170204 0.65671641791
7 0.0101071356378 0.350364963504
8 0.00778249444108 0.31884057971
9 0.00394178289873 0.329411764706
10 0.00424499696786 0.0338983050847
11 0.0040428542551 0.230769230769
12 0.00262785526582 0.0952380952381
13 0.0067717808773 0.360248447205
14 0.00616535273903 0.292134831461
15 0.0100060642814 0.506666666667
16 0.0320396199717 0.487951807229
17 0.0106124924196 0.432989690722
The weigths sum is 1.0
The computed f1-score is 0.8595133036902902
The f1-score with sklearn function is 0.85951330369029


In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)